<h2>COCO mAP</h2>

In [23]:
from collections import defaultdict
import numpy as np
from pathlib import Path
import random
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval


METHODS = {
    "VarifocalNet": "vfnet_X_101_64x4d",
    "TOOD": "tood_r101",
    "Deformable DETR": "deformable_detr_twostage_refine_r50_16x2_50e",
    "YOLOX": "yolox_x_8x8_300e",
    "Faster RCNN": "fasterrcnn_resnet50",
    "CenterNet": "centernet_resnet18_dcnv2_140e",
    "DETR": "detr_r50_8x2_150e",
    "Mask R-CNN": "mask_rcnn_x101_64x4d_fpn_mstrai_npoly_3x",
    "YOLOv3": "yolov3",
}

GT_COCO_JSON_FILE = "./annotations/annotations_person_coco_classes.json"

<h4>Let's compute COCO mAP on the entire dataset for all the considered methods</h4>

In [24]:
columns = ['Model', 'AP50', 'mAP@[0.50:0.95]']
metrics = []

for method_name, file_name in METHODS.items():
    preds_json_file = "./predictions/{}_results.json".format(file_name)
    
    # Loading coco data for eval
    cocoGt = COCO(GT_COCO_JSON_FILE)
    cocoDt = cocoGt.loadRes(preds_json_file)
    
    # Running eval
    class_ids = [1]     # we consider only the person class    
    imgIds = sorted(cocoGt.getImgIds())
    cocoEval = COCOeval(cocoGt, cocoDt, 'bbox')
    cocoEval.params.imgIds  = imgIds
    cocoEval.params.catIds = class_ids
    cocoEval.evaluate()
    cocoEval.accumulate()
    cocoEval.summarize()
    
    map = round(cocoEval.stats[0], 3)
    ap50 = round(cocoEval.stats[1], 3)
    metrics.append([method_name, ap50, map])

loading annotations into memory...
Done (t=0.72s)
creating index...
index created!
Loading and preparing results...
DONE (t=27.26s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=86.10s).
Accumulating evaluation results...
DONE (t=25.58s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.144
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.378
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.087
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.023
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.247
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.639
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.206
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.317
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxD

In [25]:
metrics_df = pd.DataFrame(metrics, columns=columns)
display(metrics_df)

,Model,AP50,mAP@[0.50:0.95]
0,VarifocalNet,0.378,0.144
1,TOOD,0.314,0.116
2,Deformable DETR,0.199,0.075
3,YOLOX,0.126,0.049
4,Faster RCNN,0.096,0.028
5,CenterNet,0.124,0.041
6,DETR,0.128,0.040
7,Mask R-CNN,0.109,0.033
8,YOLOv3,0.011,0.009


<h4>Let's now compute COCO AP50 at different altitudes for the best model</h4>

In [27]:
BEST_METHOD = {
    "VarifocalNet": "vfnet_X_101_64x4d",
}


columns = ['Altitude', 'AP50']
metrics = []

imgIds_height = defaultdict(list)
imgIds = sorted(cocoGt.getImgIds())
cocoGt = COCO(GT_COCO_JSON_FILE)

for img_id in imgIds:
    img_name = cocoGt.loadImgs(img_id)[0]['file_name']
    img_height = img_name.split("_")[3]
    imgIds_height[img_height].append(img_id)
    

for method_name, file_name in BEST_METHOD.items():
    preds_json_file = "./predictions/{}_results.json".format(file_name)
    cocoDt = cocoGt.loadRes(preds_json_file)
    cocoEval = COCOeval(cocoGt, cocoDt, 'bbox')
    for img_height, img_ids in imgIds_height.items():
        print("Evaluating images at {}".format(img_height))
        cocoEval.params.imgIds  = img_ids
        cocoEval.evaluate()
        cocoEval.accumulate()
        cocoEval.summarize()
        
        ap50 = round(cocoEval.stats[1], 3)
        metrics.append([img_height, ap50])

loading annotations into memory...
Done (t=0.71s)
creating index...
index created!
Loading and preparing results...
DONE (t=31.25s)
creating index...
index created!
Evaluating images at 30m
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=101.06s).
Accumulating evaluation results...
DONE (t=17.74s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.148
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.400
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.086
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.062
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.229
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.267
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.363
 Average Recall     (AR) @[ IoU=0.5

In [28]:
metrics_df = pd.DataFrame(metrics, columns=columns)
display(metrics_df)

,Altitude,AP50
0,30m,0.400
1,10m,0.973
2,20m,0.771
3,40m,0.540
4,50m,0.241
5,60m,0.205
